In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the datetime module from the datetime library.
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [3]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7e07de43d0daa01003fdb8c68ed96c45


In [4]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=7e07de43d0daa01003fdb8c68ed96c45&q=Boston


In [5]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [6]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print(coordinates)

[(-7.040875531836534, -76.63002014077203), (74.34500326647654, 26.308629412183137), (27.02090364598554, 157.32322245309496), (88.1986157644188, 46.07364387081614), (22.836373279898808, 176.90073400996442), (54.64840639239665, -43.36972628186115), (-70.10534252674701, 29.12542321258357), (33.36007141208579, 22.709016110002466), (-23.275669265642208, 61.70640372478789), (-74.38531340281077, -41.128371329045564), (-53.287192327890125, 29.612557571131106), (-69.33602784679081, 14.677352825596614), (69.81054408176979, 177.30807893369547), (58.74882424412465, 76.98313088599684), (-0.09632956631789114, -65.12074168100332), (-61.06724865016226, -5.135760050011925), (35.90600917924432, -148.96563795492108), (2.922407328710861, -119.96468387609437), (-66.26174145305927, -81.05071829859853), (68.33359586613338, 38.26572954611373), (-60.64590609821239, 153.73717704609788), (-21.603339915852544, -111.51054426619706), (-22.39237837232814, 110.75628888756671), (-67.1357377227913, -5.8905158000838185)

In [7]:
print(len(coordinates))

1500


In [8]:
print(coordinates[0])

(-7.040875531836534, -76.63002014077203)


In [9]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [10]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 78.28,
  'feels_like': 78.21,
  'temp_min': 67.95,
  'temp_max': 84.81,
  'pressure': 1014,
  'humidity': 51},
 'visibility': 10000,
 'wind': {'speed': 3.44, 'deg': 0},
 'clouds': {'all': 90},
 'dt': 1621725962,
 'sys': {'type': 1,
  'id': 3486,
  'country': 'US',
  'sunrise': 1621674983,
  'sunset': 1621728346},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [11]:
# Get the JSON data.
boston_data = city_weather.json()

In [12]:
boston_data["sys"]

{'type': 1,
 'id': 3486,
 'country': 'US',
 'sunrise': 1621674983,
 'sunset': 1621728346}

In [13]:
boston_data["sys"]["country"]

'US'

In [14]:
boston_data["main"]["temp_max"]

84.81

In [15]:
boston_data["coord"]["lat"]

42.3584

In [16]:
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
country = boston_data["sys"]["country"]
print(lat,lng,max_temp,humidity,clouds,wind,country)

42.3584 -71.0598 84.81 51 90 3.44 US


In [17]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-05-22 23:26:02'

In [18]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

614

In [19]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | bellavista
Processing Record 2 of Set 1 | honningsvag
Processing Record 3 of Set 1 | hasaki
Processing Record 4 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 5 of Set 1 | butaritari
Processing Record 6 of Set 1 | nanortalik
Processing Record 7 of Set 1 | port elizabeth
Processing Record 8 of Set 1 | darnah
Processing Record 9 of Set 1 | bambous virieux
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | hermanus
Processing Record 12 of Set 1 | leningradskiy
Processing Record 13 of Set 1 | strezhevoy
Processing Record 14 of Set 1 | santa isabel do rio negro
Processing Record 15 of Set 1 | cape town
Processing Record 16 of Set 1 | kahului
Processing Record 17 of Set 1 | san patricio
Processing Record 18 of Set 1 | punta arenas
Processing Record 19 of Set 1 | ostrovnoy
Processing Record 20 of Set 1 | hobart
Processing Record 21 of Set 1 | rikitea
Processi

Processing Record 36 of Set 4 | tasiilaq
Processing Record 37 of Set 4 | constitucion
Processing Record 38 of Set 4 | ponta do sol
Processing Record 39 of Set 4 | saint-philippe
Processing Record 40 of Set 4 | cascais
Processing Record 41 of Set 4 | norman wells
Processing Record 42 of Set 4 | louisbourg
City not found. Skipping...
Processing Record 43 of Set 4 | bathsheba
Processing Record 44 of Set 4 | waipawa
Processing Record 45 of Set 4 | calvinia
Processing Record 46 of Set 4 | khatanga
Processing Record 47 of Set 4 | yatou
Processing Record 48 of Set 4 | acapulco
Processing Record 49 of Set 4 | baraboo
Processing Record 50 of Set 4 | yumen
Processing Record 1 of Set 5 | tshela
Processing Record 2 of Set 5 | grindavik
Processing Record 3 of Set 5 | prince rupert
Processing Record 4 of Set 5 | mathbaria
Processing Record 5 of Set 5 | chuy
Processing Record 6 of Set 5 | baruun-urt
Processing Record 7 of Set 5 | bam
Processing Record 8 of Set 5 | babushkin
Processing Record 9 of Set

Processing Record 25 of Set 8 | synya
Processing Record 26 of Set 8 | pangnirtung
Processing Record 27 of Set 8 | mao
Processing Record 28 of Set 8 | beringovskiy
Processing Record 29 of Set 8 | antiguo morelos
Processing Record 30 of Set 8 | port hawkesbury
Processing Record 31 of Set 8 | cap-aux-meules
Processing Record 32 of Set 8 | macroom
Processing Record 33 of Set 8 | mareeba
Processing Record 34 of Set 8 | aksarka
Processing Record 35 of Set 8 | hunchun
Processing Record 36 of Set 8 | novikovo
Processing Record 37 of Set 8 | mbandaka
Processing Record 38 of Set 8 | dodge city
Processing Record 39 of Set 8 | kutum
Processing Record 40 of Set 8 | bajo baudo
City not found. Skipping...
Processing Record 41 of Set 8 | azanka
Processing Record 42 of Set 8 | nelson bay
Processing Record 43 of Set 8 | mtwara
Processing Record 44 of Set 8 | sechura
Processing Record 45 of Set 8 | ulaangom
Processing Record 46 of Set 8 | puerto madryn
Processing Record 47 of Set 8 | ayagoz
Processing Re

Processing Record 11 of Set 12 | mount darwin
Processing Record 12 of Set 12 | vila velha
Processing Record 13 of Set 12 | waynesboro
Processing Record 14 of Set 12 | urumqi
Processing Record 15 of Set 12 | mabay
Processing Record 16 of Set 12 | denpasar
Processing Record 17 of Set 12 | bonavista
Processing Record 18 of Set 12 | christchurch
Processing Record 19 of Set 12 | praia da vitoria
Processing Record 20 of Set 12 | copperas cove
Processing Record 21 of Set 12 | laguna
Processing Record 22 of Set 12 | klyuchi
Processing Record 23 of Set 12 | dakar
Processing Record 24 of Set 12 | bouna
Processing Record 25 of Set 12 | gidole
Processing Record 26 of Set 12 | tenosique
Processing Record 27 of Set 12 | deputatskiy
Processing Record 28 of Set 12 | khonuu
City not found. Skipping...
Processing Record 29 of Set 12 | tshikapa
Processing Record 30 of Set 12 | pontianak
Processing Record 31 of Set 12 | safwah
City not found. Skipping...
Processing Record 32 of Set 12 | vestmannaeyjar
Pro

In [20]:
len(city_data)

558

### Convert array of dictionary to pandas DataFrame

In [21]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bellavista,-7.0561,-76.5911,85.28,85,42,2.42,PE,2021-05-22 23:29:31
1,Honningsvag,70.9821,25.9704,34.81,57,100,16.28,NO,2021-05-22 23:29:31
2,Hasaki,35.7333,140.8333,64.80,82,99,15.52,JP,2021-05-22 23:29:32
3,Butaritari,3.0707,172.7902,83.66,80,100,18.30,KI,2021-05-22 23:29:32
4,Nanortalik,60.1432,-45.2371,34.66,73,75,7.99,GL,2021-05-22 23:29:32
5,Port Elizabeth,-33.9180,25.5701,49.10,76,0,3.44,ZA,2021-05-22 23:29:33
6,Darnah,32.7670,22.6367,63.82,73,9,10.04,LY,2021-05-22 23:24:44
7,Bambous Virieux,-20.3428,57.7575,69.22,71,75,18.01,MU,2021-05-22 23:29:33
8,Ushuaia,-54.8000,-68.3000,37.06,93,40,2.30,AR,2021-05-22 23:29:34
9,Hermanus,-34.4187,19.2345,53.91,62,8,7.34,ZA,2021-05-22 23:22:36


In [24]:
new_column_order = ["City", "Country", "Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]

In [25]:
new_df = city_data_df[new_column_order]
new_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Bellavista,PE,2021-05-22 23:29:31,-7.0561,-76.5911,85.28,85,42,2.42
1,Honningsvag,NO,2021-05-22 23:29:31,70.9821,25.9704,34.81,57,100,16.28
2,Hasaki,JP,2021-05-22 23:29:32,35.7333,140.8333,64.80,82,99,15.52
3,Butaritari,KI,2021-05-22 23:29:32,3.0707,172.7902,83.66,80,100,18.30
4,Nanortalik,GL,2021-05-22 23:29:32,60.1432,-45.2371,34.66,73,75,7.99
5,Port Elizabeth,ZA,2021-05-22 23:29:33,-33.9180,25.5701,49.10,76,0,3.44
6,Darnah,LY,2021-05-22 23:24:44,32.7670,22.6367,63.82,73,9,10.04
7,Bambous Virieux,MU,2021-05-22 23:29:33,-20.3428,57.7575,69.22,71,75,18.01
8,Ushuaia,AR,2021-05-22 23:29:34,-54.8000,-68.3000,37.06,93,40,2.30
9,Hermanus,ZA,2021-05-22 23:22:36,-34.4187,19.2345,53.91,62,8,7.34


In [26]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
